## Import stuff

In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

print(tf.__version__)

from tensorflow.python.client import device_lib
print("tensorflow doesn't detects GPU: ")
print(device_lib.list_local_devices() )
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())

print("torch detects GPU: " , torch.cuda.is_available())

oversampler = RandomOverSampler(random_state=0)
undersampler = ClusterCentroids(random_state=0)


def get_score(df_true, df_submission):
    for i in range(df_true.shape[1]):
        print("score for col ",i,": " , metrics.roc_auc_score(df_true[:,i], df_submission[:,i]))


2.3.0
tensorflow doesn't detects GPU: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2780746865416498767
]

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
torch detects GPU:  True


## Load training data from csv


In [8]:
features_df_normalized = pd.read_csv("train_features_normalized.csv")
labels_df = pd.read_csv("train_labels.csv")

X = np.array(features_df_normalized.iloc[:, 2:].values,  dtype=float).reshape(227940//12,35*12)


X_train = X[:int(X.shape[0]*0.8)]
X_valid = X[int(X.shape[0]*0.8):]


Y = np.array(labels_df.iloc[:, 1:].values,  dtype=float)
Y_train = Y[:int(Y.shape[0]*0.8)]
Y_valid = Y[int(Y.shape[0]*0.8):]


print("X shape: ", X.shape)
print("Y shape: ", Y.shape)


X shape:  (18995, 420)
Y shape:  (18995, 15)


## The function that trains and rates models

In [18]:
def find_best_model(col, X_train, Y_train, X_valid, Y_valid, X_test):
    models = []
    print("training nn for col ", col)
    nn = tf.keras.Sequential([
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    nn.compile(optimizer='adam', loss='binary_crossentropy')
    nn.fit(X_train, Y_train[:,col], epochs=10, verbose=False)
    models.append(nn)

    if(col<11):
        print("training lr for col ", col)
        lr = LogisticRegression(max_iter=1000)
        lr.fit(X_train, Y_train[:,col])
        models.append(lr)
    
    print("done training for col ", col)

    best_score = 0
    best_model = None
    for model in models:
        pred = model.predict(X_valid)
        if(col>10):
            score = 0.5 + 0.5 * np.maximum(0, metrics.r2_score(Y_valid[:,col], pred))
        else:
            score = metrics.roc_auc_score(Y_valid[:,col], pred)
        print("score for",type(model),": " , score)
        if score > best_score:
            best_score = score
            best_model = model

    print("best model: ", type(best_model))
    predictions = best_model.predict(X_test)

    return predictions

## Writing results

In [ ]:

X_test = pd.read_csv("test_features_normalized.csv")
X_test = np.array(X_test.iloc[:, 2:].values,  dtype=float).reshape(151968//12,35*12)

for i in range(0,15):
    pred = find_best_model(i, X_train, Y_train, X_valid, Y_valid, X_test)

# print(pred)


# pred1 = []
# for i in range(0, 15):
#     Y_pred_lr = pred1.append(models[i].predict_proba(test_X_normalized)[:,1])


# pred1 = np.array(pred1).transpose()
# # pred1 = model1.predict(test_X_normalized)
# pred2 = model2.predict(test_X_normalized)
# # pred2 = model2.predict(test_X_small)
# # pred2 = (pred2+1)/2
# # np.clip(pred2, 0, 1, out=pred2)
# pred3 = model3.predict(test_X_normalized)

# print(pred2[pred2==0].shape)
# print(pred2[pred2>0.5].shape)


In [ ]:
# sample = pd.read_csv("sample.csv")
# print(sample.shape)
# submit =  pd.DataFrame(np.zeros((12664,16)), columns=[sample.columns])
# print(submit.shape)
# print(pred1.shape)
# print(pred2.shape)
# print(pred3.shape)
# # print(submit)


# submit.iloc[:,0] = sample.iloc[:,0]
# submit.iloc[:,1:11] = pred1
# submit.iloc[:,11] = pred2
# submit.iloc[:,12:] = pred3

# submit.to_csv('submit.zip', index=False, float_format='%.3f', compression='zip')
# submit.to_csv('submit.csv', index=False, float_format='%.3f')

(12664, 16)
(12664, 16)
(12664, 10)
(12664,)
(12664, 4)
